In [19]:
from utils_stim import get_story_wordseqs
import argparse
import json
import config
import os
import numpy as np
import pickle

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument("--sessions", nargs = "+", type = int, 
    default = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 18, 20])
args = parser.parse_args("")

# training stories
stories = []
with open(os.path.join(config.DATA_TRAIN_DIR, "sess_to_story.json"), "r") as f:
    sess_to_story = json.load(f) 
for sess in args.sessions:
    stories.extend(sess_to_story[str(sess)])

In [11]:
word_seqs = get_story_wordseqs(stories)

In [17]:
ds = word_seqs['souls']

In [30]:
documents = [ds.data for ds in word_seqs.values()]

In [77]:
trimmed_documents = []
MAXLEN = 512

for d in documents:
    if len(d) <= MAXLEN:
        trimmed_documents.append(" ".join(d))
    else:
        splits = int(np.ceil(len(d) / MAXLEN))
        for i in range(splits):
            trimmed_documents.append(" ".join(d[i*MAXLEN: (i+1)*MAXLEN]))


In [78]:
sum([len(d) for d in trimmed_documents])

766633

In [83]:
with open('textgrid_dataset.pickle', 'wb') as file:
    pickle.dump(trimmed_documents, file)

In [20]:
with open('textgrid_dataset.pickle', 'rb') as file:
    trimmed_documents = pickle.load(file)

In [21]:
import transformers

In [22]:
gpt2_tokenizer = transformers.GPT2Tokenizer.from_pretrained('distilbert/distilgpt2', eos_token=None, bos_token=None, pad_token='<pad>', unk_token='<unk>', add_prefix_space=False)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
gpt2_tokenizer.encode('enne d ')

[29727, 288, 220]

In [24]:
tokenized_documents = [gpt2_tokenizer.tokenize(d) for d in trimmed_documents]

In [26]:
sum([len(d) for d in tokenized_documents])

165678